In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
def filtering_company(company):
    if pd.isna(company):
        return np.nan
    
    original = company
    company = company.replace('(농업회사법인유한회사)', '')
    company = company.replace('의료법인)', '')
    company = company.replace('농업회사법인', '')
    company = company.replace('가부시키가이샤', '')
    company = company.replace('카부시키카이샤', '')
    company = company.replace('(유한회사)', '')
    company = company.replace('주식회사', '')
    company = company.replace('㈜', '')
    company = company.replace('(주)', '')
    company = company.replace('（주）', '')
    company = company.replace('( 주 )', '')
    company = company.replace('( 주)', '')
    company = company.replace('(주 )', '')
    company = company.replace('주)', '')
    company = company.replace('주 )', '')
    company = company.replace('(유)', '')
    company = company.replace('유)', '')
    company = company.replace('우)', '')
    company = company.replace('(사)', '')
    company = company.replace('사)', '')
    company = company.replace('  ', ' ') # 신정보개발주식회사 ~~
    company = company.split('|')[0] # 상위 -> 하위

    try:
        
        length = len(company)
        index = set(range(length))

        pairs1 = list()
        stack = list()
        for i in range(length):
            if company[i] == '(':
                stack.append(i)
            elif company[i] == ')':
                if stack:
                    pairs1.append((stack.pop(), i))
                else:
                    if pairs1:
                        pairs1.append((pairs1.pop()[0], i))
                    else:
                        pairs1.append((i, i))
        
        if stack:
            pairs1.append((stack[0], length-1))
        
        pairs2 = list()
        stack = list()
        for i in range(length):
            if company[i] == '[':
                stack.append(i)
            elif company[i] == ']':
                if stack:
                    pairs2.append((stack.pop(), i))
                else:
                    if pairs2:
                        pairs2.append((pairs2.pop()[0], i))
                    else:
                        pairs2.append((i, i))
                        
        if stack:
            pairs2.append((stack[0], length-1))
            
        for start, end in pairs1 + pairs2:
            index -= set(range(start, end+1))

        index = list(index)
        index.sort()
        company = ''.join([company[idx] for idx in index])
        company = company.strip()
    
    except:
        print(original, '--->', company)
        
    return company

In [3]:
biz = pd.read_csv('../../playground/biz2common_final.tsv', sep='\t')
biz.head()

,index,회사명,사업자번호
0,0,1004마트,4188601752
1,1,100물류,6198107320
2,2,100케이션,4358601742
3,3,100홈,2892700667
4,4,1020와이오엔지4지아이엘,1018674570


In [4]:
kotc_value = pd.read_csv('./kotc_value.csv')
kotc_value = kotc_value.rename(columns={'name':'회사명'})
kotc_value['value'] = kotc_value['value'] * 100000000

In [6]:
boosting_predict = pd.read_csv('./predict/boosting_predict.csv')
DecisionTree_predict = pd.read_csv('./predict/boosting_DecisionTreeRegressor.csv')
ElasticNet_predict = pd.read_csv('./predict/boosting_ElasticNet.csv')
Lass_predict = pd.read_csv('./predict/boosting_Lasso.csv')
LGBM_predict = pd.read_csv('./predict/boosting_LGBMRegressor.csv')
RandomForest_predict = pd.read_csv('./predict/boosting_RandomForestRegressor.csv')
Ridge_predict = pd.read_csv('./predict/boosting_Ridge.csv')
XGBM_predict = pd.read_csv('./predict/boosting_XGBRegressor.csv')
market_predict = pd.read_csv('./predict/market_value_method.csv')
lgb_optuna = pd.read_csv('./predict/boosting_lgb_optuna.csv')
grid_lgb = pd.read_csv('./predict/grid_lgbm.csv')
lstm_value = pd.read_csv('./predict/8_pred (1).csv')

In [7]:
grid_lgb = grid_lgb[['회사명', 'predict']].rename(columns={'predict':'predict_grid_lgb'})
lstm_value = lstm_value.rename(columns={'기업 가치': 'lstm_value'})

In [8]:
boosting_predict = boosting_predict[['회사명', 'predict']]
market_predict = market_predict[['회사명_비상장', '평균 기업가치']]
market_predict = market_predict.rename(columns={'회사명_비상장':'회사명', '평균 기업가치':'predict'})

In [9]:
boosting_predict = boosting_predict.merge(kotc_value, on='회사명')

In [10]:
merged = boosting_predict.merge(market_predict, on='회사명')
merged = merged.merge(DecisionTree_predict, on='회사명')
merged = merged.merge(ElasticNet_predict, on='회사명')
merged = merged.merge(Lass_predict, on='회사명')
merged = merged.merge(LGBM_predict, on='회사명')
merged = merged.merge(RandomForest_predict, on='회사명')
merged = merged.merge(Ridge_predict, on='회사명')
merged = merged.merge(XGBM_predict, on='회사명')
merged = merged.merge(lgb_optuna, on='회사명')
merged = merged.merge(grid_lgb, on='회사명')
merged = merged.merge(lstm_value, on='회사명')
merged = merged.drop_duplicates('회사명').reset_index(drop=True)


In [11]:
cols = ['회사명', 'value', 'predict_x', 'predict_y','predict_DecisionTreeRegressor', 'predict_ElasticNet', 'predict_Lasso','predict_LGBMRegressor', 'predict_RandomForestRegressor','predict_Ridge', 'predict_XGBRegressor', 'predict', 'predict_grid_lgb', 'lstm_value']
merged = merged[cols]

In [12]:
merged

,회사명,value,predict_x,predict_y,predict_DecisionTreeRegressor,predict_ElasticNet,predict_Lasso,predict_LGBMRegressor,predict_RandomForestRegressor,predict_Ridge,predict_XGBRegressor,predict,predict_grid_lgb,lstm_value
0,아리바이오,482300000000,1.560328e+11,1.105818e+11,1.632703e+11,6.088211e+10,6.088211e+10,2.003248e+11,1.183812e+11,6.088211e+10,2.948207e+11,2.243913e+11,1.192124e+11,1.094928e+10
1,조인트리,20800000000,7.555491e+10,8.180875e+10,1.019384e+11,8.766692e+10,8.766692e+10,1.091422e+11,9.600022e+10,8.766692e+10,1.047221e+11,9.983949e+10,1.166856e+11,-6.702875e+09
2,코스텔,20300000000,6.173794e+10,7.579700e+10,6.013424e+10,7.981873e+10,7.981873e+10,5.313956e+10,6.314528e+10,7.981873e+10,5.735212e+10,8.374779e+10,1.037027e+11,-2.295043e+10
3,비보존,209300000000,9.260856e+10,4.543395e+10,5.690368e+10,5.722570e+10,5.722570e+10,9.542641e+10,6.589283e+10,5.722570e+10,1.110532e+11,6.274299e+10,1.042602e+11,7.560594e+10
4,동우당제약,1700000000,3.908414e+10,7.312594e+10,3.724240e+10,6.804550e+10,6.804550e+10,5.122271e+10,2.770498e+10,6.804550e+10,6.509121e+10,3.474297e+10,1.018022e+11,9.690027e+09
5,오리엔트전자,17600000000,3.956924e+10,5.944804e+10,6.013424e+10,6.873761e+10,6.873761e+10,4.460525e+10,4.859957e+10,6.873761e+10,6.001004e+10,6.573185e+10,1.018022e+11,-4.181383e+10
6,익수제약,27600000000,6.186929e+10,8.235344e+10,5.628357e+10,8.251598e+10,8.251598e+10,2.774983e+10,5.638709e+10,8.251598e+10,3.258129e+10,5.754368e+10,1.068601e+11,6.643345e+09
7,아이월드제약,48900000000,3.835535e+10,8.524074e+10,3.724240e+10,6.475213e+10,6.475213e+10,5.332419e+10,2.865181e+10,6.475213e+10,6.112448e+10,3.368196e+10,1.018022e+11,-1.696244e+10
8,한국지네틱바이오팜,5000000000,2.895342e+10,2.306699e+10,1.937125e+10,6.897471e+10,6.897471e+10,1.767337e+10,2.399314e+10,6.897471e+10,3.419446e+10,1.736606e+10,9.206022e+10,1.699082e+10
9,뉴젠홀딩스,10800000000,2.895342e+10,1.063276e+11,1.937125e+10,6.888985e+10,6.888985e+10,1.567302e+10,2.404243e+10,6.888985e+10,3.988735e+10,1.736606e+10,9.206022e+10,-3.059573e+09


# rmse

In [13]:
model_list = []
mae_list = []
rmse_list = []
for model in merged.columns[2:]:
    print(model)
    model_list.append(model)
    rmse_list.append(np.sqrt(mean_squared_error(merged[model], merged['value'])))
    mae_list.append(mean_absolute_error(merged[model], merged['value']))
    print(rmse_list[-1])
    print(mae_list[-1])
    print('================')

predict_x
97483281393.4235
57527955028.11835
predict_y
130901746642.6672
90436006757.11191
predict_DecisionTreeRegressor
100023274560.44493
57977649911.057205
predict_ElasticNet
130590686101.6769
86542523225.79
predict_Lasso
130590686101.67432
86542523225.79031
predict_LGBMRegressor
87577424686.0297
50368444638.943504
predict_RandomForestRegressor
109092955513.11662
61126250815.89136
predict_Ridge
130590686149.35526
86542523141.59361
predict_XGBRegressor
68056590757.70245
48909758512.42857
predict
86595483102.97604
54267193493.38373
predict_grid_lgb
126810899180.49187
101729632918.1884
lstm_value
134229999607.36061
64154479708.25654


In [14]:
for i in range(len(mae_list)):
    print('=========================')
    print(model_list[i])
    print(mae_list[i])
    print(rmse_list[i])
    print('=========================')

predict_x
57527955028.11835
97483281393.4235
predict_y
90436006757.11191
130901746642.6672
predict_DecisionTreeRegressor
57977649911.057205
100023274560.44493
predict_ElasticNet
86542523225.79
130590686101.6769
predict_Lasso
86542523225.79031
130590686101.67432
predict_LGBMRegressor
50368444638.943504
87577424686.0297
predict_RandomForestRegressor
61126250815.89136
109092955513.11662
predict_Ridge
86542523141.59361
130590686149.35526
predict_XGBRegressor
48909758512.42857
68056590757.70245
predict
54267193493.38373
86595483102.97604
predict_grid_lgb
101729632918.1884
126810899180.49187
lstm_value
64154479708.25654
134229999607.36061


In [15]:
min(mae_list)

48909758512.42857

In [16]:
model_list

['predict_x',
 'predict_y',
 'predict_DecisionTreeRegressor',
 'predict_ElasticNet',
 'predict_Lasso',
 'predict_LGBMRegressor',
 'predict_RandomForestRegressor',
 'predict_Ridge',
 'predict_XGBRegressor',
 'predict',
 'predict_grid_lgb',
 'lstm_value']

In [17]:
mae_list

[57527955028.11835,
 90436006757.11191,
 57977649911.057205,
 86542523225.79,
 86542523225.79031,
 50368444638.943504,
 61126250815.89136,
 86542523141.59361,
 48909758512.42857,
 54267193493.38373,
 101729632918.1884,
 64154479708.25654]

In [116]:
merged.head(1)

,회사명,value,predict_x,predict_y,predict_DecisionTreeRegressor,predict_ElasticNet,predict_Lasso,predict_LGBMRegressor,predict_RandomForestRegressor,predict_Ridge,predict_XGBRegressor,predict,predict_grid_lgb,lstm_value
0,아리바이오,482300000000,1.560328e+11,1.105818e+11,1.632703e+11,6.088211e+10,6.088211e+10,2.003248e+11,1.183812e+11,6.088211e+10,2.948207e+11,2.243913e+11,1.192124e+11,2.740152e+11


In [117]:
merged.corr()

<ipython-input-117-c6ad84058222>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged.corr()


,value,predict_x,predict_y,predict_DecisionTreeRegressor,predict_ElasticNet,predict_Lasso,predict_LGBMRegressor,predict_RandomForestRegressor,predict_Ridge,predict_XGBRegressor,predict,predict_grid_lgb,lstm_value
value,1.000000,0.932759,0.486622,0.803549,-0.561469,-0.561469,0.894868,0.723781,-0.561469,0.950177,0.856203,0.618624,0.638965
predict_x,0.932759,1.000000,0.600005,0.913531,-0.268212,-0.268212,0.939560,0.904523,-0.268212,0.934213,0.930750,0.813125,0.598580
predict_y,0.486622,0.600005,1.000000,0.714805,0.097359,0.097359,0.635027,0.605684,0.097359,0.597219,0.703553,0.770548,0.629704
predict_DecisionTreeRegressor,0.803549,0.913531,0.714805,1.000000,-0.036464,-0.036464,0.933434,0.954373,-0.036464,0.912646,0.980982,0.919566,0.723434
predict_ElasticNet,-0.561469,-0.268212,0.097359,-0.036464,1.000000,1.000000,-0.303864,0.094693,1.000000,-0.407604,-0.128003,0.181433,-0.074549
predict_Lasso,-0.561469,-0.268212,0.097359,-0.036464,1.000000,1.000000,-0.303864,0.094693,1.000000,-0.407604,-0.128003,0.181433,-0.074549
predict_LGBMRegressor,0.894868,0.939560,0.635027,0.933434,-0.303864,-0.303864,1.000000,0.880389,-0.303864,0.974218,0.925673,0.840385,0.628822
predict_RandomForestRegressor,0.723781,0.904523,0.605684,0.954373,0.094693,0.094693,0.880389,1.000000,0.094693,0.816647,0.918117,0.924610,0.536861
predict_Ridge,-0.561469,-0.268212,0.097359,-0.036464,1.000000,1.000000,-0.303864,0.094693,1.000000,-0.407604,-0.128003,0.181433,-0.074549
predict_XGBRegressor,0.950177,0.934213,0.597219,0.912646,-0.407604,-0.407604,0.974218,0.816647,-0.407604,1.000000,0.935119,0.754806,0.732288


In [118]:
mean_absolute_error(merged['predict'], merged['value'])

71294112261.30226

In [119]:
mean_absolute_error(merged['predict'] * 0.5 + merged['predict_y'] * 0.5, merged['value'])

79380270804.329

In [17]:
for a, b, c in ((3, 4, 3), (5, 5, 0), (0, 5, 5), (0, 8, 2)):
    print(a, b, c)
    print(mean_absolute_error((merged['lstm_value'] * a + merged['predict'] * b + merged['predict_y'] * c) / 10, merged['value']))

3 4 3
68960726391.57784
5 5 0
61431246864.159325
0 5 5
79380270804.329
0 8 2
73683667840.34314


In [18]:
# LGBM_predict = pd.read_csv('./predict/boosting_LGBMRegressor.csv')
market_predict = pd.read_csv('./predict/market_value_method.csv')
lgb_optuna = pd.read_csv('./predict/boosting_lgb_optuna.csv')
lstm_value = pd.read_csv('./predict/4_pred.csv')
lstm_value = pd.read_csv('./predict/8_pred (1).csv')

In [19]:
lstm_value.head(1)

,회사명,기업 가치
0,공공공간,-1.683022e+08


In [20]:
lgb_optuna[lgb_optuna['회사명'] == '케이조선']

,회사명,업종명,주생산품,year,부채총액,현금,자본총계,매출액,영업이익(EBIT),감가상각비,당기순이익,순부채,상각전영업이익(EBITDA),상장분류,사업자번호,predict


In [21]:
lgb_optuna.head(1)

,회사명,업종명,주생산품,year,부채총액,현금,자본총계,매출액,영업이익(EBIT),감가상각비,당기순이익,순부채,상각전영업이익(EBITDA),상장분류,사업자번호,predict
0,공공공간,제품 디자인업,디자인 제품,2022,422356000,200714000,415272000,609411000,41242000,0,32351000,221642000,41242000,False,101-86-76618,1.749146e+10


In [22]:
print(len(market_predict), len(lgb_optuna), len(lstm_value))

23639 27511 10838


In [23]:
merged = lgb_optuna.merge(lstm_value, on='회사명')

In [24]:
merged['ensemble'] = merged['predict'] * 0.5 + merged['기업 가치'] * 0.5

In [128]:
not_listed = lgb_optuna[~lgb_optuna['회사명'].isin(merged['회사명'])]

In [129]:
merged['사업자번호'] = merged['사업자번호'].apply(lambda x: x.replace('-', ''))

In [130]:
not_listed['사업자번호'] = not_listed['사업자번호'].apply(lambda x: x.replace('-', ''))

<ipython-input-130-b7520200ff56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_listed['사업자번호'] = not_listed['사업자번호'].apply(lambda x: x.replace('-', ''))


In [131]:
merged.head(1)

,회사명,업종명,주생산품,year,부채총액,현금,자본총계,매출액,영업이익(EBIT),감가상각비,당기순이익,순부채,상각전영업이익(EBITDA),상장분류,사업자번호,predict,기업 가치,ensemble
0,공공공간,제품 디자인업,디자인 제품,2022,422356000,200714000,415272000,609411000,41242000,0,32351000,221642000,41242000,False,1018676618,1.749146e+10,-2.323058e+08,8.629578e+09


In [132]:
biz = pd.read_csv('./biz2common (2).tsv', sep='\t')

In [133]:
biz

,index,회사명,사업자번호
0,0,1004마트,4188601752
1,1,100물류,6198107320
2,2,100케이션,4358601742
3,3,100홈,2892700667
4,4,1020와이오엔지4지아이엘,1018674570
...,...,...,...
128637,119125,힘찬장애인복지회,4098217913
128638,119126,힘찬홀딩스,1228198986
128639,119127,힘테크,6198700824
128640,119128,힘펠,1248179802


In [134]:
up = merged[['회사명', 'ensemble']].rename(columns={'ensemble':'predict'})

In [135]:
down = not_listed[['회사명', 'predict']]

In [136]:
ensembled = pd.concat([up, down])

In [137]:
ensembled

,회사명,predict
0,공공공간,8.629578e+09
1,신정보개발,7.751706e+09
2,이시스비젼,1.088484e+10
3,부산육가공센터,1.414431e+10
4,덴플렉스,6.987200e+09
...,...,...
27505,글로이스,1.986035e+10
27506,직스테크놀로지,1.815770e+10
27507,파워플레이어,1.579368e+10
27508,오늘,1.671043e+10


In [138]:
listed = pd.read_csv('./230725시총.csv')
listed.head(1)

,날짜,시가총액,종목코드,회사명
0,2023-07-25,120856238580,60310.0,3S


In [87]:
listed[listed['회사명'] == '케이조선']

,날짜,시가총액,종목코드,회사명


In [140]:
listed = listed.merge(biz, on='회사명')

In [143]:
listed[listed['회사명'] == '삼성전자']

,날짜,시가총액,종목코드,회사명,index,사업자번호
1032,2023-07-25,417884778500000,5930.0,삼성전자,38323,1248100998


In [142]:
listed = listed.drop_duplicates('index')

In [144]:
listed_merge = listed[['회사명', '시가총액']].rename(columns={'시가총액':'value'})

In [145]:
listed_merge

,회사명,value
0,3S,120856238580
1,AJ네트웍스,196419527525
2,AP시스템,303336206850
3,AP위성,174954726400
4,BGF리테일,2760239788200
...,...,...
2902,흥구석유,75600000000
2903,흥국,81576247520
2905,흥국에프엔비,103354904525
2906,흥아해운,520519906335


In [149]:
ensembled = ensembled.rename(columns={'predict':'value'})

In [150]:
ensembled

,회사명,value
0,공공공간,8.629578e+09
1,신정보개발,7.751706e+09
2,이시스비젼,1.088484e+10
3,부산육가공센터,1.414431e+10
4,덴플렉스,6.987200e+09
...,...,...
27505,글로이스,1.986035e+10
27506,직스테크놀로지,1.815770e+10
27507,파워플레이어,1.579368e+10
27508,오늘,1.671043e+10


In [152]:
final_merged = pd.concat([ensembled[['회사명', 'value']], listed_merge])

In [153]:
final_merged[final_merged['회사명'] == '삼성전자']

,회사명,value
1032,삼성전자,4.178848e+14


In [159]:
temp = final_merged.merge(biz, on='회사명')

In [162]:
temp = temp.drop_duplicates('index')

In [163]:
temp[temp['회사명'] == '삼성전자']

,회사명,value,index,사업자번호
34644,삼성전자,4.178848e+14,38323,1248100998


In [164]:
temp.to_csv('./ensemble_value.csv')

In [167]:
temp[temp['회사명'] == '삼성전자']

,회사명,value,index,사업자번호
34644,삼성전자,4.178848e+14,38323,1248100998


In [80]:
ensembled[ensembled['회사명'] == '케이조선']

,회사명,value,index,사업자번호


In [55]:
lstm_value1 = pd.read_csv('./predict/4_pred.csv')
lstm_value1 = lstm_value1[lstm_value1['기업 가치'] > 0]
lstm_value2 = pd.read_csv('./predict/5_pred.csv')
lstm_value2 = lstm_value2[lstm_value2['기업 가치'] > 0]
lstm_value3 = pd.read_csv('./predict/6_pred.csv')
lstm_value3 = lstm_value3[lstm_value3['기업 가치'] > 0]
lstm_value4 = pd.read_csv('./predict/8_pred (1).csv')
lstm_value4 = lstm_value4[lstm_value4['기업 가치'] > 0]

In [56]:
lstm_compare = lstm_value1.merge(lstm_value2, on='회사명')
lstm_compare = lstm_compare.rename(columns={'기업 가치_x': '1', '기업 가치_y': '2'})
lstm_compare = lstm_compare.merge(lstm_value3, on='회사명')
lstm_compare = lstm_compare.merge(lstm_value4, on='회사명')

In [57]:
lstm_compare = lstm_compare.merge(kotc_value, on='회사명')

In [58]:
lstm_compare

,회사명,1,2,기업 가치_x,기업 가치_y,value
0,아리바이오,2.740152e+11,2.740152e+11,2.857776e+10,1.094928e+10,482300000000
1,익수제약,9.570823e+09,9.825251e+09,9.804744e+09,6.643345e+09,27600000000
2,한국지네틱바이오팜,5.802423e+09,5.802423e+09,3.170388e+09,1.699082e+10,5000000000


In [59]:
for col in lstm_compare.columns[1:-1]:
    print(col)
    print(mean_absolute_error(lstm_compare[col], lstm_compare['value']))

1
75705461738.01567
2
75620652174.01959
기업 가치_x
157782369188.3259
기업 가치_y
168099397514.1425
